In [32]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/document.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

memory = ConversationBufferMemory(return_messages=True)

def load_memory(_):
    x = memory.load_memory_variables({})
    return {"history": x["history"]}


final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Given the following extracted parts of a long document and a question, create a final answer. 
            If you don't know the answer, just say that you don't know. Don't try to make up an answer.
            ------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

chain = {"context": retriever, "question": RunnablePassthrough()} | final_prompt | llm

chain.invoke("Is Aaronson guilty?")


AIMessage(content='Yes, according to the document, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.')

In [33]:
chain.invoke("What message did he write in the table?")

AIMessage(content='In the document, the message he wrote on the table was "2+2=5".')

In [34]:
chain.invoke("Who is Julia?")

AIMessage(content='Julia is a character from the document who is associated with the protagonist in a complex relationship.')